#### Link to storage site and prepare the notebook

Link current notebook to Google Drive to access the data

In [ ]:
import os, sys 
from google.colab import drive
drive.mount('/content/drive')

Download the latest codes from github, install it, and add the package into the working directory (might need to restart the notebook as asked - won't need to do this one your computer)

In [ ]:
!pip install --ignore-installed git+https://github.com/samsam2610/AnalysisSupport
import sys
sys.path.append("/AnalysisSupport")

Use the built-in functions to install neccessary packages for the project.
- Would be good idea to have separate virtual environments for each type of project

In [ ]:
# Setup required packages
from setup_package import *
# setup_package.project_3d() # for 3D/multi-cameras projects
setup_package.project_2d() # for projects using only single camera

#### Process the 2D videos and projects

Install the required packages to work on 2D videos and projects

In [ ]:
# Setup required packages
from setup_package import *
setup_package.project_2d() # for projects using only single camera

Import DeepLabCut and sets the parameters for headless operations (no GUI)

In [ ]:
import os
import sys
os.environ["DLClight"]="True"

import deeplabcut
deeplabcut.__version__

Specify the project name, video type, and the directory to the config file.

In [ ]:
ProjectFolderName = 'Spinal Implant-Sam-2021-10-05'
VideoType = 'avi' 

path_config_file = '/content/drive/MyDrive/Spinal Implant-Sam-2021-10-05/config.yaml'

This step takes a look at the current directory and the config file for the current stage of the project. If the model is already trained and is being improved upon, it sets the initial weight for the model using the weight from the latest trained model in the directory. 

It also sets the directory to the inital weight to appropriate path based on current directory. This is to prevent error when the project is trained on the cloud and modified locally and vice versa.

In [ ]:
from dlc_support import *
setConfig_ContinueTraining(path_config_file, 0)

Once prepared, we can start training the network using the built-in function.

In [ ]:
deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10, saveiters=500, maxiters=3000000)

Create list of directories of videos you want to process

In [ ]:
videofile_pathlist = [ 
                      '/content/drive/MyDrive/Rat/SCI tests/Julie/07292021Julie',
                      '/content/drive/MyDrive/Rat/SCI tests/Julie/08052021&08062021Julie/0805JULIETetanic',
                      '/content/drive/MyDrive/Rat/SCI tests/Julie/08052021&08062021Julie/0806JULIESinglePulse',
                      '/content/drive/MyDrive/Rat/SCI tests/Julie/08112021Julie',
                      '/content/drive/MyDrive/Rat/SCI tests/Julie/08222021Julie'
                     ]

Once trained, the network can then be used to label the video. This step creates `ProcessVideoList` object from the list of paths above. For each path, the following steps are taken:
- Check for processing status of videos in the folder. If not already create, it creates a cvs file holding the processing status. If the video is already processed and no significant error is found. The processing status can be changed manually. Once changed, the manager reads it and excludes the videos from the processing list. This step saves time from preprocessing the good videos over and over again.
- This generates a list of unprocessed videos

In [ ]:
from dlc_support import *
videoListObject = ProcessVideoList(videofile_pathlist) # add comment
videoUnprocessList = videoListObject.getListofUnprocessedVideos()
print(videoUnprocessList)

Using the list of unprocessed videos, these following steps process those video, plot the trajectory, and create the labeled video.

In [ ]:
deeplabcut.analyze_videos(path_config_file, videoUnprocessList, videotype=VideoType, save_as_csv=True)
deeplabcut.plot_trajectories(path_config_file, videoUnprocessList, videotype=VideoType)
deeplabcut.create_labeled_video(path_config_file, videoUnprocessList, videotype=VideoType)

#### Process the 3D projects

Install the required packages to work on 3D projects

In [ ]:
from setup_package import *
setup_package.project_3d() # for 3D/multi-cameras projects

Import the library and preprare the list of projects that you want to process by their directories

Note: if you have only one project, still put the directory in the list (square bracket).




In [ ]:
from anipose_support import *
videopath_list = ['/content/drive/MyDrive/Rat/Treadmill test /rat-e/12-6/vids']

This step creates a ```PathManager``` object, as the name indicated, that handles all the intermediate steps that you wish to do to your projects. By creating a path manager, the following operations are done under the hood:
- The manager looks for all the videos with the specified type in `videotype` in the directory
- The manager then pairs the videos based on the specified prefix `cam_names`
- Then, it specifies the calibrations based on the calibration indicator  `calib_name` within the name.
- If not moves already, the manager moves the calibration videos into the subfolder `calibration` within the directory. If the calibration videos are not found in the base directory, it would look for the videos in the `calibration` folder.
- Next, for each pair of NON-calibration videos, the path manager creates a `ProjectManager` object and pairs it with the found calibration videos.
- Upon created, the `ProjectManager` 
 - Create `config.toml` file if it is not already there.
 - Checks for file generated by the calibration process. If the project is already calibrated, it loads the calibration parameters into the object. Else, it starts the calibration process, loads the parameter, and generates the file.
 - Once loaded, the manager labels the calibration videos based on the calibration parameters to double check the process.
 - (more steps to be added)
 - Phew! Done! Standby waiting for next order!


In [ ]:
projects_path_list = PathManager(videopath_list, videotype='.avi', cam_names=['cam1', 'cam2'], calib_name ='calib')
# projects_list = projects_path_list.get_projects_list()
# print(projects_list)

Once the list is initialized properly, the following processes can be apply to all the projects or individual project (not shown here):
- `ProjectManger.process_triangulate` - `PathManager.batch_triagulate`: this process first check for labeled data by `DeepLabCut`. If the files are not in the directory. It skips this step.
- `ProjectManager.export_data` - `PathManager.batch_export_data`: this process exports the data triangulated in the above process.
- `ProjectManager.plot_data` - `PathManager.batch_plot_data`: this process plots the triangulated in the above process. This one still needs some works.

In [ ]:
projects_path_list.batch_triangulate()
projects_path_list.batch_export_data()
projects_path_list.batch_plot_data()